In [10]:
import json
from pathlib import Path

import pandas as pd
import numpy as np

from bw2io.importers.io import IOImporter
from bw2io.strategies.io import tidy_tables

In [2]:
pet_hiot = pd.DataFrame([[1,-2],[0,1]], # experiment setting 1 prod to 0
             index=pd.MultiIndex.from_tuples([('DK','prod1'),('DK','prod2')]),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

pet_hiot = pet_hiot.astype(pd.SparseDtype("float",0))


B = pd.DataFrame([[1,3],[1,2],[0,1]],
             index=pd.Index(['co2_air','ch4_air','co2_accelerated_air']),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

B = B.astype(pd.SparseDtype("float",0))

fd = pd.DataFrame.from_dict(
    {
    ('DK','Household'):{('DK','prod1'):-11,('DK','prod2'):-3},
    ('DK','Government'):{('DK','prod1'):-8,('DK','prod2'):-4},
    ('DK','Capital'):{('DK','prod1'):-4,('DK','prod2'):-2}
        }
    )

Bfd = pd.DataFrame([[1,3],],
index=pd.Index(['co2_air',]),
columns = pd.MultiIndex.from_tuples([('DK','Household'),
('DK','Government')])).astype(pd.SparseDtype("float",0))

In [3]:
print(pet_hiot)
print(B)
print(fd)
print(Bfd)

           DK     
         act1 act2
DK prod1  1.0 -2.0
   prod2    0  1.0
                      DK     
                    act1 act2
co2_air              1.0  3.0
ch4_air              1.0  2.0
co2_accelerated_air    0  1.0
                DK                   
         Household Government Capital
DK prod1       -11         -8      -4
   prod2        -3         -4      -2
               DK           
        Household Government
co2_air       1.0        3.0


In [4]:
pfd = pd.DataFrame((np.eye(fd.shape[1])),index=fd.columns,columns=fd.columns)
fd_total = pd.concat([fd,pfd])
fd_total = fd_total.astype(pd.SparseDtype("float",0))

In [5]:
extended_hiot = pd.concat([pet_hiot,fd_total],axis=1).fillna(0)
extended_B = pd.concat([B,Bfd],axis=1).fillna(0)

In [6]:
print(extended_hiot)

                DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0


In [7]:
print(extended_B)

                      DK                          
                    act1 act2 Household Government
co2_air              1.0  3.0       1.0        3.0
ch4_air              1.0  2.0       0.0        0.0
co2_accelerated_air    0  1.0       0.0        0.0


In [ ]:
path_to_intermediate = (Path.cwd()/'results')
path_to_intermediate.mkdir(exist_ok=True)

In [21]:
metadata_dict = {'prod1':{'unit':'kg','name':'product 1'},
                 'prod2':{'unit':'kg','name':'product 2'},
                 'Household':{'unit':'unit','name':'the household'},
                 'Government':{'unit':'unit','name':'the government'},
                 'Capital':{'unit':'unit','name':'capital investments'},

                 'co2_air':{'unit':'ton', # not standard units
                            'name':'carbon dioxide',
                            'compartment':('air',)},
                 'ch4_air':{'unit':'kg',
                            'name':'methane',
                            'compartment':('air',)},
                 'co2_accelerated_air':{'unit':'kg', # additional biosphere flow
                                        'name':'carbon dioxide accelerated',
                                        'compartment':('air',)}
                }

with open(path_to_intermediate/'io_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp,indent=4)

In [20]:
tidy_tables(extended_hiot,extended_B,path_to_intermediate)